<a href="https://colab.research.google.com/github/shives-tkt/FINAL-PRO2/blob/main/PRo_2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score

# Step 1: Load the dataset
data_url = 'https://drive.google.com/uc?id=1VLrUOjudactyRQD8KLlLrds7DwXXa7H1'
data = pd.read_csv(data_url)

# Step 2: Explore the data
print(data.head())
print(data.info())
print(data.describe())

# Step 3: Handle missing values
data.fillna(data.median(), inplace=True)

# Step 4: Encode categorical variables
label_encoders = {}
for col in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Step 5: Normalize numerical features
scaler = StandardScaler()
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
scaled_data = scaler.fit_transform(data[numerical_cols])
data[numerical_cols] = scaled_data

# Step 6: Perform clustering
kmeans = KMeans(n_clusters=5, random_state=42)
data['Cluster'] = kmeans.fit_predict(data[numerical_cols])

# Validate clusters
print("Silhouette Score:", silhouette_score(data[numerical_cols], data['Cluster']))

# Visualize clusters
sns.pairplot(data, hue='Cluster')
plt.show()

# Step 7: Train regression models
X = data.drop(columns=['Yield', 'Cluster'])  # Replace 'Yield' with the target column name if different
y = data['Yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Step 8: Visualize results
# Example: Plot actual vs predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel("Actual Yield")
plt.ylabel("Predicted Yield")
plt.title("Actual vs Predicted Yield")
plt.show()

# Step 9: Save the model and create documentation
import joblib
joblib.dump(model, 'crop_rotation_model.pkl')